In [ ]:
import sys
from pathlib import Path
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq
import numpy as np
from google.cloud import bigquery

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

In [91]:
%reload_ext autoreload
%autoreload 2

In [93]:
from datetime import datetime, timedelta

# Obtener la fecha de hoy
hoy = datetime.now()

# Calcular el último lunes (inicio de la semana pasada)
ultimo_lunes = (hoy - timedelta(days=hoy.weekday() + 7)).replace(hour=0, minute=0, second=0, microsecond=0)

# Calcular el último domingo (fin de la semana pasada)
ultimo_domingo = ultimo_lunes + timedelta(days=6)

print(f"Última semana completa: {ultimo_lunes.date()} a {ultimo_domingo.date()}")


Última semana completa: 2025-08-04 a 2025-08-10


In [ ]:
from src.data_utils import load_raw_data

# Calcular el último lunes y domingo (ya calculados arriba, pero los reutilizamos aquí)
# Descargar y cargar solo los datos de esa semana (flujo integrado)
df = load_raw_data(
    fecha_inicio=ultimo_lunes.strftime("%Y-%m-%d"),
    fecha_fin=ultimo_domingo.strftime("%Y-%m-%d"),
    descargar_bq=True
 )

print("Primeras filas de la semana cargada:")
print(df.head())